In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install metaphone

In [ ]:
pip install fuzzymatcher

In [ ]:
import re

In [ ]:
import pandas as pd
import numpy as np
import datetime as DT
from datetime import date
from pathlib import Path
import fuzzymatcher

In [ ]:
DATA_PATH = 'data'
phhs = pd.read_csv(DATA_PATH + '/Geocoded_Appointment_Data_RiskStratified_2020-04-11_05-05-03.csv')
hhsc = pd.read_csv(DATA_PATH + '/geocoded_positive_puis_2020-04-10.csv')

In [ ]:
phhs['DOB'] = pd.to_datetime(phhs['DOB'], errors='coerce')
now = pd.to_datetime('now')
phhs['Age'] = (now - phhs['DOB']).astype('<m8[Y]') 

In [ ]:
phhs['Age']= phhs['Age'].astype(int)

In [ ]:
hhsc_copy = hhsc.replace(to_replace = "F",
                  value = "Female")

In [ ]:
hhsc_copy = hhsc_copy.replace(to_replace = "M",
                  value = "Male")

In [ ]:
hhsc_copy['Age (Yrs) at presentation'] = hhsc_copy['Age (Yrs) at presentation'].map(lambda x:re.sub(r'\W+', '',x))

In [ ]:
hhsc_copy['Age (Yrs) at presentation'] = hhsc_copy['Age (Yrs) at presentation'].astype(int)

In [ ]:
hhsc_copy['Age (Yrs) at presentation'].apply(type)

In [ ]:
phhs['PHHS_Lookup'] = phhs[['geoaddress', 'Sex'
]].apply(lambda x: '_'.join(x), axis=1)

hhsc_copy['HHSC_Lookup'] = hhsc_copy[['geoaddress', 'Sex',
]].apply(lambda x: '_'.join(x), axis=1)


In [ ]:
left_on = ["PHHS_Lookup", "Age"]
right_on = ["HHSC_Lookup", "Age (Yrs) at presentation"]

In [ ]:
match_results = fuzzymatcher.fuzzy_left_join(phhs, 
                                             hhsc_copy, 
                                             left_on, 
                                             right_on)

In [ ]:
match_results[['addnum_left','rest1']] = match_results.geoaddress_left.str.split(n=1,expand=True)

In [ ]:
match_results[['addnum_right','rest2']] = match_results.geoaddress_right.str.split(n=1,expand=True)

In [ ]:
match_results['addnum_left'] = match_results['addnum_left'].apply(pd.to_numeric, errors = 'coerce')

In [ ]:
match_results ['addnum_right'] = match_results['addnum_right'].apply(pd.to_numeric, errors = 'coerce')

In [ ]:
match_results['Match'] = np.select([(match_results['Age'] == match_results['Age (Yrs) at presentation']) & (match_results['best_match_score'] >= 0.8) & (match_results['Sex_left'] == match_results['Sex_right']) & (match_results['addnum_left'] == match_results['addnum_right'])]
                     ,['Y'], default = 'N')

In [ ]:
match_results = match_results[["MRN", "Match", "Specimen Collection Date"
]]

In [ ]:
match_results.to_csv('data/match_results.csv', index = False)